# Transforming Complex Data Types in Spark SQL

In this notebook we're going to go through some data transformation examples using Spark SQL. Spark SQL supports many
built-in transformation functions in the module `pyspark.sql.functions` therefore we will start off by importing that.

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Convenience function for turning JSON strings into DataFrames.
def jsonToDataFrame(json, schema=None):
  # SparkSessions are available with Spark 2.0+
  reader = spark.read
  if schema:
    reader.schema(schema)
  return reader.json(sc.parallelize([json]))

<b>Selecting from nested columns</b> - Dots (`"."`) can be used to access nested columns for structs and maps.

In [4]:
# Using a struct
schema = StructType().add("a", StructType().add("b", IntegerType()))
                          
events = jsonToDataFrame("""
{
  "a": {
     "b": 1
  }
}
""", schema)

display(events.select("a.b"))

b
1.0


In [5]:
# Using a map
schema = StructType().add("a", MapType(StringType(), IntegerType()))
                          
events = jsonToDataFrame("""
{
  "a": {
     "b": 1
  }
}
""", schema)

display(events.select("a.b"))

b
1.0


<b>Flattening structs</b> - A star (`"*"`) can be used to select all of the subfields in a struct.

In [7]:
events = jsonToDataFrame("""
{
  "a": {
     "b": 1,
     "c": 2
  }
}
""")

display(events.select("a.*"))

b,c
1.0,2.0


<b>Nesting columns</b> - The `struct()` function or just parentheses in SQL can be used to create a new struct.

In [9]:
events = jsonToDataFrame("""
{
  "a": 1,
  "b": 2,
  "c": 3
}
""")

display(events.select(struct(col("a").alias("y")).alias("x")))

x
List(1.0)


<b>Nesting all columns</b> - The star (`"*"`) can also be used to include all columns in a nested struct.

In [11]:
events = jsonToDataFrame("""
{
  "a": 1,
  "b": 2
}
""")

display(events.select(struct("*").alias("x")))

x
"List(1.0, 2.0)"


<b>Selecting a single array or map element</b> - `getItem()` or square brackets (i.e. `[ ]`) can be used to select a single element out of an array or a map.

In [13]:
events = jsonToDataFrame("""
{
  "a": [1, 2]
}
""")

display(events.select(col("a").getItem(0).alias("x")))

x
1.0


In [14]:
# Using a map
schema = StructType().add("a", MapType(StringType(), IntegerType()))

events = jsonToDataFrame("""
{
  "a": {
    "b": 1
  }
}
""", schema)

display(events.select(col("a").getItem("b").alias("x")))

x
1.0


<b>Creating a row for each array or map element</b> - `explode()` can be used to create a new row for each element in an array or each key-value pair.  This is similar to `LATERAL VIEW EXPLODE` in HiveQL.

In [16]:
events = jsonToDataFrame("""
{
  "a": [1, 2]
}
""")

display(events.select(explode("a").alias("x")))

x
1.0
2.0


In [17]:
# Using a map
schema = StructType().add("a", MapType(StringType(), IntegerType()))

events = jsonToDataFrame("""
{
  "a": {
    "b": 1,
    "c": 2
  }
}
""", schema)

display(events.select(explode("a").alias("x", "y")))

x,y
b,1.0
c,2.0


<b>Collecting multiple rows into an array</b> - `collect_list()` and `collect_set()` can be used to aggregate items into an array.

In [19]:
events = jsonToDataFrame("""
[{ "x": 1 }, { "x": 2 }]
""")

display(events.select(collect_list("x").alias("x")))

x
"List(1.0, 2.0)"


In [20]:
# using an aggregation
events = jsonToDataFrame("""
[{ "x": 1, "y": "a" }, { "x": 2, "y": "b" }]
""")

display(events.groupBy("y").agg(collect_list("x").alias("x")))

y,x
b,List(2.0)
a,List(1.0)


<b>Selecting one field from each item in an array</b> - when you use dot notation on an array we return a new array where that field has been selected from each array element.

In [22]:
events = jsonToDataFrame("""
{
  "a": [
    {"b": 1},
    {"b": 2}
  ]
}
""")

display(events.select("a.b"))

b
"List(1.0, 2.0)"


<b>Convert a group of columns to json</b> - `to_json()` can be used to turn structs into json strings. This method is particularly useful when you would like to re-encode multiple columns into a single one when writing data out to Kafka. This method is not presently available in SQL.

In [24]:
events = jsonToDataFrame("""
{
  "a": {
    "b": 1
  }
}
""")

display(events.select(to_json("a").alias("c")))

c
"{""b"":1}"


<b>Parse a column containing json</b> - `from_json()` can be used to turn a string column with json data into a struct. Then you may flatten the struct as described above to have individual columns. This method is not presently available in SQL. 
**This method is available since Spark 2.1**

In [26]:
events = jsonToDataFrame("""
{
  "a": "{\\"b\\":1}"
}
""")

schema = StructType().add("b", IntegerType())
display(events.select(from_json("a", schema).alias("c")))

c
List(1.0)


Sometimes you may want to leave a part of the JSON string still as JSON to avoid too much complexity in your schema.

In [28]:
events = jsonToDataFrame("""
{
  "a": "{\\"b\\":{\\"x\\":1,\\"y\\":{\\"z\\":2}}}"
}
""")

schema = StructType().add("b", StructType().add("x", IntegerType())
                            .add("y", StringType()))
display(events.select(from_json("a", schema).alias("c")))

c
"List(List(1.0, {""z"":2}))"


<b>Parse a set of fields from a column containing json</b> - `json_tuple()` can be used to extract a fields available in a string column with json data.

In [30]:
events = jsonToDataFrame("""
{
  "a": "{\\"b\\":1}"
}
""")

display(events.select(json_tuple("a", "b").alias("c")))

c
1


<b>Parse a well formed string column</b> - `regexp_extract()` can be used to parse strings using regular expressions.

In [32]:
events = jsonToDataFrame("""
[{ "a": "x: 1" }, { "a": "y: 2" }]
""")

display(events.select(regexp_extract("a", "([a-z]):", 1).alias("c")))

c
x
y
